In [1]:
import sys
sys.path.insert(0, '../src')
import harmoutil

import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit

from keras.models import Model
from keras.layers import Input, Dense, GRU, concatenate

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.models import load_model

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Load data
raw_data = harmoutil.load_pickled_data("../data/refined_data.pkl") # lists of (chord label, melody seqs) by sections
print("---Raw data---")
print("Number of sections: {}".format(len(raw_data)))
print("Sample section: {}\n".format(raw_data[0]))
augmented_data = harmoutil.transpose_and_augment_data(raw_data)
print("---Transpose and augment data---")
print("Number of sections after data augmentation: {}".format(len(augmented_data)))
print("Sample section: {}\n".format(augmented_data[0]))
data = [harmoutil.to_sevenths(section) for section in augmented_data]
print("---Truncate chords to sevenths---")
print("Number of sections: {}".format(len(data)))
print("Sample section: {}\n".format(data[0]))
data = [harmoutil.melody_to_octave_range(section) for section in data]
print("---Convert melody to integers---")
print("Number of sections: {}".format(len(data)))
print("Sample section: {}\n".format(data[0]))

---Raw data---
Number of sections: 2408
Sample section: [('Bb6', [[58.0], [58.0]]), ('G7', [[-1.0], [-1.0]]), ('C-7', [[-1.0], [-1.0]]), ('F7', [[-1.0], [-1.0]]), ('Bb', [[-1.0], [-1.0]]), ('G-7', [[50.0], [57.0, 60.0]]), ('C-7', [[58.0, 55.0], [58.0]]), ('F7', [[61.0], [60.0, 58.0]]), ('F-7', [[60.0], [58.0]]), ('Bb7', [[56.0, 60.0], [59.0, 57.0]]), ('Eb7', [[58.0, 54.0], [55.0, 58.0]]), ('Ab7', [[61.0, 56.0], [61.0, 62.0]]), ('D-7', [[58.0, 60.0], [55.0, 58.0]]), ('G7', [[58.0], [-1.0]]), ('C-7', [[-1.0], [-1.0]]), ('F7', [[-1.0], [-1.0]])]

---Transpose and augment data---
Number of sections after data augmentation: 28884
Sample section: [('E6', [[52.0], [52.0]]), ('Db7', [[-1.0], [-1.0]]), ('Gb-7', [[-1.0], [-1.0]]), ('B7', [[-1.0], [-1.0]]), ('E', [[-1.0], [-1.0]]), ('Db-7', [[44.0], [51.0, 54.0]]), ('Gb-7', [[52.0, 49.0], [52.0]]), ('B7', [[55.0], [54.0, 52.0]]), ('B-7', [[54.0], [52.0]]), ('E7', [[50.0, 54.0], [53.0, 51.0]]), ('A7', [[52.0, 48.0], [49.0, 52.0]]), ('D7', [[55.0, 

In [3]:
# Create individual chord and melody element lists 
def get_notes_by_chord(beats):
    return [note for beat in beats for note in beat]

def get_chords_by_section(section):
    return [chord_info[0] for chord_info in section]

def check_if_augmented_major(section):
    section_chords = get_chords_by_section(section)
    for ch in section_chords:
        if "+j7" in ch:
            return True
    return False


# Remove sections that involve augmented major chords (since not enough data to even allow StratifiedShuffleSplit)
data = [section for section in data if not check_if_augmented_major(section)]
print("---Remove sections with augmented major chord---")
print("Number of sections: {}\n".format(len(data)))

chords_by_sections = [get_chords_by_section(section) for section in data]
chords = [chord_info[0] for section in data for chord_info in section]
notes_by_chords = [get_notes_by_chord(chord_info[1]) for section in data for chord_info in section]
notes = [note for chord_notes in notes_by_chords for note in chord_notes]
# print(sum([len(section) for section in chords_by_sections]))
print("Number of sections: {} | Sample section chords: {}".format(len(chords_by_sections), chords_by_sections[0]))
print("Number of chords: {} | Sample chord: {}".format(len(chords), chords[0]))
print("Number of melodies {} | Sample melody: {}".format(len(notes_by_chords), notes_by_chords[0]))
print("Number of melody notes in the data: {} | Sample melody note: {}".format(len(notes), notes[0]))

---Remove sections with augmented major chord---
Number of sections: 28836

Number of sections: 28836 | Sample section chords: ['E6', 'Db7', 'Gb-7', 'B7', 'E', 'Db-7', 'Gb-7', 'B7', 'B-7', 'E7', 'A7', 'D7', 'Ab-7', 'Db7', 'Gb-7', 'B7']
Number of chords: 333480 | Sample chord: E6
Number of melodies 333480 | Sample melody: [4, 4]
Number of melody notes in the data: 2195328 | Sample melody note: 4


In [4]:
# Create categorical data mappings

# Note to integer index
note_to_int = dict([(c, i) for i, c in enumerate(sorted(list(set(notes)))[1:])])
note_to_int[-1] = len(note_to_int)
note_to_int['<pad>'] = len(note_to_int)
print("Melody note to integer mapping:\n {}\n".format(note_to_int))

# Integer to note
int_to_note = dict([(k, v) for v, k in note_to_int.items()])
print("Integer to melody note mapping:\n {}\n".format(int_to_note))

# Chord to integer index
chord_to_int = dict([(c, i) for i, c in enumerate(sorted(list(set(chords))))])
chord_to_int['<bos>'] = len(chord_to_int)
print("Chord label to integer mapping:\n {}\n".format(chord_to_int))


# Integer to chord index
int_to_chord = dict([(k, v) for v, k in chord_to_int.items()])
print("Integer to chord label mapping:\n {}\n".format(int_to_chord))

Melody note to integer mapping:
 {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, '<pad>': 13, -1: 12}

Integer to melody note mapping:
 {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: -1, 13: '<pad>'}

Chord label to integer mapping:
 {'Bo7': 57, 'F': 135, 'Abm7b5': 19, 'Db6': 91, 'D': 75, 'Dj7': 99, 'Fo7': 147, 'Gbj7': 168, 'Gb': 159, 'Gsus7': 179, 'D7': 83, 'Ab-7': 14, 'Fsus': 148, 'G': 150, 'Bb-6': 43, 'Gb-7': 164, 'A7': 8, 'D6': 82, 'F+7': 137, 'Db+7': 86, 'Bbsus7': 53, 'G6': 157, 'Bb': 39, 'Eb-j7': 120, 'Cm7b5': 70, 'Eb-': 117, 'Abo': 20, 'G-': 153, 'Dbm7b5': 94, 'Do7': 102, 'F+': 136, 'Gb-6': 163, 'A-7': 5, 'Bbj7': 48, 'B-j7': 36, 'C': 60, 'B-': 33, 'Bb+': 40, 'Fj7': 144, 'A-j7': 6, 'B': 30, 'A': 0, 'Bb-7': 44, 'Dbsus': 97, 'Esus': 133, 'Bsus7': 59, 'Ab-j7': 15, 'C6': 67, 'Absus': 22, 'C+7': 62, 'E+': 106, 'B-7': 35, 'B+': 31, 'Dbsus7': 98, 'Dbo7': 96, 'Fm7b5': 145, 'D-j7': 81, 'Bb-j7': 45, 'Dsus7': 104, 'Aj7': 24, 'Db

In [5]:
# Define numerical variables

n_samples = len(chords)
n_notes = len(note_to_int)
n_chords = len(chord_to_int)
n_pitches = 12
max_melody_len = max([len(mel_seq) for mel_seq in notes_by_chords])
chord_context_len = 7

print("Total number of samples: {}".format(n_samples))
print("Number of distinct melody notes: {}".format(n_notes))
print("Number of distinct chord labels: {}".format(n_chords))
print("Number of distinct chord pitches: {}".format(n_pitches))
print("Maximum melody sequence length: {}".format(max_melody_len))
print("Fixed context chord sequence length: {}".format(7))

Total number of samples: 333480
Number of distinct melody notes: 14
Number of distinct chord labels: 182
Number of distinct chord pitches: 12
Maximum melody sequence length: 135
Fixed context chord sequence length: 7


In [6]:
# Prepare tensor data

def pad_melody(melody, max_len):
    return melody + (max_len-len(melody))*['<pad>']

def build_input_chord_sequences(chord_seq, context_len):
    padded_sequence = context_len*['<bos>'] + chord_seq
    formatted_sequences = [padded_sequence[i:i+context_len+1] for i in range(len(chord_seq))]
    return formatted_sequences

# Melody
input_melody_data = [pad_melody(melody, max_melody_len) for melody in notes_by_chords]
print("Sample input melody sequence: {}\n".format(input_melody_data[5]))


# Chords
formatted_chords_data = []
for section_chords in chords_by_sections:
    formatted_chords_data += build_input_chord_sequences(section_chords, chord_context_len)

input_chords_data = [ch[:-1] for ch in formatted_chords_data]
target_chords_data = [ch[-1] for ch in formatted_chords_data]
print("Sample input chord sequence: {}\n".format(input_chords_data[5]))
print("Sample target chord: {}\n".format(target_chords_data[5]))

print("Input melody: {}, Input chords: {}, Target chords: {}".format(len(input_melody_data), len(input_chords_data), len(target_chords_data)))

Sample input melody sequence: [8, 3, 6, '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

In [7]:
# Build tensors

X_melody = np.zeros((n_samples, max_melody_len, n_notes), dtype='float32')
X_chords = np.zeros((n_samples, chord_context_len, n_pitches), dtype='float32')
Y = np.zeros((n_samples, n_pitches), dtype='float32')
Y_labels = np.zeros((n_samples, n_chords), dtype='float32')

for i, (input_mel, input_ch, target_ch) in enumerate(zip(input_melody_data, input_chords_data, target_chords_data)):
    
    Y_labels[i, chord_to_int[target_ch]] = 1 # tensor of chords one-hot vectors used for dataset splitting
    
    target_chord_pitches = [0]*12 if target_ch == '<bos>' else harmoutil.chord_to_notes(target_ch)
    for j, pitch_presence in enumerate(target_chord_pitches):
        Y[i, j] = pitch_presence

    for j, chord in enumerate(input_ch):
        input_chord_pitches = [0]*12 if chord == '<bos>' else harmoutil.chord_to_notes(chord)
        for k, pitch_presence in enumerate(input_chord_pitches):
            X_chords[i, j, k] = pitch_presence
            
    # Added below...
    for j, note in enumerate(input_mel):
        X_melody[i, j, note_to_int[note]] = 1

In [8]:
# Split dataset into 80%-10%-10% train-valid-test

seed = 0

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

for train_index, aux_index in sss.split(X_chords, Y_labels):
    X_melody_train, X_melody_aux = X_melody[train_index], X_melody[aux_index]
    X_chords_train, X_chords_aux = X_chords[train_index], X_chords[aux_index]
    Y_train, Y_aux = Y[train_index], Y[aux_index]
    Y_labels_train, Y_labels_aux = Y_labels[train_index], Y_labels[aux_index]
    
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=seed)

for valid_index, test_index in sss.split(X_chords_aux, Y_labels_aux):
    X_melody_valid, X_melody_test = X_melody[valid_index], X_melody[test_index]
    X_chords_valid, X_chords_test = X_chords[valid_index], X_chords[test_index]
    Y_valid, Y_test = Y[valid_index], Y[test_index]
    Y_labels_valid, Y_labels_test = Y_labels[valid_index], Y_labels[test_index]

In [9]:
# Define neural net architecture

latent_dim = 128

melody_input = Input(shape=(max_melody_len, n_notes))
melody_gru = GRU(latent_dim)(melody_input)

chords_input = Input(shape=(chord_context_len, n_pitches))
chords_gru = GRU(latent_dim)(chords_input)

concat = concatenate([melody_gru, chords_gru])

chord_dense = Dense(n_pitches, activation='sigmoid')(concat)

model = Model([melody_input, chords_input], chord_dense)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())
# SVG(model_to_dot(model).create(prog='dot', format='svg'))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 135, 14)       0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 7, 12)         0                                            
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 128)           54912       input_1[0][0]                    
____________________________________________________________________________________________________
gru_2 (GRU)                      (None, 128)           54144       input_2[0][0]                    
___________________________________________________________________________________________

In [10]:
# Introduce Early-Stopping and Save-Best-Performance callbacks
filepath = "../models/pitch-Mel1-Cho1-FC1_150ep_melo.h5"
bp = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [11]:
# Train model
batch_size = 256
epochs = 150

history = model.fit([X_melody_train, X_chords_train], Y_train, epochs=epochs, validation_data=([X_melody_valid, X_chords_valid], Y_valid,), batch_size=batch_size, callbacks=[bp])

Train on 266784 samples, validate on 33348 samples
Epoch 1/150
266784/266784 [==============================] - 189s - loss: 0.5597 - acc: 0.7218 - val_loss: 0.5225 - val_acc: 0.7513
Epoch 2/150
266784/266784 [==============================] - 189s - loss: 0.5067 - acc: 0.7614 - val_loss: 0.4857 - val_acc: 0.7753
Epoch 3/150
266784/266784 [==============================] - 185s - loss: 0.4775 - acc: 0.7788 - val_loss: 0.4658 - val_acc: 0.7860
Epoch 4/150
266784/266784 [==============================] - 192s - loss: 0.4585 - acc: 0.7898 - val_loss: 0.4526 - val_acc: 0.7939
Epoch 5/150
266784/266784 [==============================] - 194s - loss: 0.4440 - acc: 0.7981 - val_loss: 0.4426 - val_acc: 0.7991
Epoch 6/150
266784/266784 [==============================] - 187s - loss: 0.4322 - acc: 0.8051 - val_loss: 0.4350 - val_acc: 0.8039
Epoch 7/150
266784/266784 [==============================] - 187s - loss: 0.4223 - acc: 0.8104 - val_loss: 0.4295 - val_acc: 0.8070
Epoch 8/150
266784/266784

266784/266784 [==============================] - 189s - loss: 0.3186 - acc: 0.8621 - val_loss: 0.3837 - val_acc: 0.8370
Epoch 54/150
266784/266784 [==============================] - 184s - loss: 0.3182 - acc: 0.8623 - val_loss: 0.3861 - val_acc: 0.8365
Epoch 55/150
266784/266784 [==============================] - 192s - loss: 0.3188 - acc: 0.8619 - val_loss: 0.3849 - val_acc: 0.8373
Epoch 56/150
266784/266784 [==============================] - 192s - loss: 0.3181 - acc: 0.8624 - val_loss: 0.3883 - val_acc: 0.8356
Epoch 57/150
266784/266784 [==============================] - 196s - loss: 0.3185 - acc: 0.8621 - val_loss: 0.3851 - val_acc: 0.8366
Epoch 58/150
266784/266784 [==============================] - 181s - loss: 0.3163 - acc: 0.8631 - val_loss: 0.3861 - val_acc: 0.8373
Epoch 59/150
266784/266784 [==============================] - 192s - loss: 0.3158 - acc: 0.8636 - val_loss: 0.3840 - val_acc: 0.8381
Epoch 60/150
266784/266784 [==============================] - 192s - loss: 0.3161 

266784/266784 [==============================] - 186s - loss: 0.3060 - acc: 0.8679 - val_loss: 0.3789 - val_acc: 0.8415
Epoch 113/150
266784/266784 [==============================] - 196s - loss: 0.3053 - acc: 0.8682 - val_loss: 0.3805 - val_acc: 0.8413
Epoch 114/150
266784/266784 [==============================] - 184s - loss: 0.3073 - acc: 0.8671 - val_loss: 0.3815 - val_acc: 0.8408
Epoch 115/150
266784/266784 [==============================] - 186s - loss: 0.3102 - acc: 0.8656 - val_loss: 0.3775 - val_acc: 0.8423
Epoch 116/150
266784/266784 [==============================] - 188s - loss: 0.3075 - acc: 0.8670 - val_loss: 0.3764 - val_acc: 0.8426
Epoch 117/150
266784/266784 [==============================] - 193s - loss: 0.3059 - acc: 0.8679 - val_loss: 0.3773 - val_acc: 0.8428
Epoch 118/150
266784/266784 [==============================] - 179s - loss: 0.3055 - acc: 0.8681 - val_loss: 0.3789 - val_acc: 0.8422
Epoch 119/150
266784/266784 [==============================] - 189s - loss: 

In [1]:
# Visualize model learning
plt.clf()

plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.tight_layout()
plt.suptitle("Training history of root model", fontsize=16)
plt.subplots_adjust(top=0.85)
plt.show()

NameError: name 'plt' is not defined

In [13]:
### Load best performance model ###
model = load_model("../models/pitch-Mel1-Cho1-FC1_150ep_melo.h5")

In [14]:
# Evaluate predictions in terms of labels

# Predict chords from each test sample melody
Y_pred = model.predict([X_melody_test, X_chords_test])

# Compute accuracy and kappa score
print("Categorical accuracy of combined chord prediction: {0:.4f}".format(harmoutil.compute_multiclass_binary_accuracy_score(Y_test, Y_pred)))
print("Kappa score of combined chord prediction: {0:.4f}".format(harmoutil.compute_multiclass_binary_kappa_score(Y_test, Y_pred)))

Categorical accuracy of combined chord prediction: 0.8406
TP: 89565 TN: 246808 FP: 23242 FN: 40561
Kappa score of combined chord prediction: 0.6237
